In [4]:
!pip install pandas scikit-learn matplotlib seaborn dhanhq

In [ ]:
import os

# Load credentials from environment variables
DHAN_CLIENT_ID = os.environ.get('DHAN_CLIENT_ID', "YOUR_CLIENT_ID_HERE")      # For Sandbox, this is 2510263737
DHAN_ACCESS_TOKEN = os.environ.get('DHAN_ACCESS_TOKEN', "YOUR_ACCESS_TOKEN_HERE") # The long token you generated
"""
programmatic_analysis_dhan.py

Full & final script:
- Robust DhanHQ intraday fetcher (tries multiple method names)
- Normalize responses to OHLCV DataFrame
- Anomaly detection via IsolationForest
- Aggregate option volumes and flag suspicious periods
- Plot and save final chart

Usage:
  python programmatic_analysis_dhan.py

Credentials are loaded from environment variables DHAN_CLIENT_ID and DHAN_ACCESS_TOKEN.
Set these in your .env file or environment. DO NOT hardcode real credentials.
"""

import os
import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

# Optional: If you have the dhanhq SDK installed
try:
    from dhanhq import dhanhq
except Exception:
    dhanhq = None

# ---------------------------
# Configuration (update me)
# ---------------------------

# Example: "eyJhbGciOiJ..." - DO NOT commit tokens to shared repos

TARGET_DATE = "2024-01-17"   # date to analyze (YYYY-MM-DD)
STOCK_NAME = "BANKNIFTY"     # label for saved files

# Security IDs — update to the correct ones if you have a symbol master
INSTRUMENTS = {
    'spot': {
        'security_id': '26001',     # example Nifty Bank index id
        'exchange_segment': 'NSE_IDX'
    },
    'futures': {
        'security_id': '50123',     # example BANKNIFTY future id
        'exchange_segment': 'NSE_FNO'
    },
    'options': [
        {'security_id': '86683', 'strike': 48000, 'type': 'PE'},
        {'security_id': '86682', 'strike': 47500, 'type': 'PE'},
        {'security_id': '86681', 'strike': 47000, 'type': 'PE'},
    ]
}

# Fetching controls
INTERVAL_MINUTES = 5
SLEEP_BETWEEN_CALLS = 1
MAX_RETRIES = 2

# Candidate Dhan client method names to try (keeps compatibility across SDK versions)
CANDIDATE_METHODS = [
    'intraday_daily_minute_charts',
    'intraday_minute_charts',
    'intraday_minutes',
    'get_intraday_minute_charts',
    'minute_charts',
    'historical_minute_charts',
    'historical_minute',
    'get_minute_data',
    'historical'
]

Dhan client created. Inspecting available methods (sample):
['BO', 'BSE', 'BSE_FNO', 'BUY', 'CNC', 'CO', 'COMPACT_CSV_URL', 'CUR', 'DAY', 'DETAILED_CSV_URL', 'FNO', 'INDEX', 'INTRA', 'IOC', 'LIMIT', 'MARGIN', 'MARKET', 'MCX', 'MTF', 'NSE', 'NSE_FNO', 'SELL', 'SL', 'SLM', 'access_token', 'base_url', 'cancel_forever', 'cancel_order', 'client_id', 'convert_position', 'convert_to_date_time', 'disable_ssl', 'edis_inquiry', 'expiry_list', 'fetch_security_list', 'generate_tpin', 'get_forever', 'get_fund_limits', 'get_holdings', 'get_order_by_correlationID', 'get_order_by_id', 'get_order_list', 'get_positions', 'get_trade_book', 'get_trade_history', 'header', 'historical_daily_data', 'intraday_minute_data', 'kill_switch', 'ledger_report', 'margin_calculator', 'modify_forever', 'modify_order', 'ohlc_data', 'open_browser_for_tpin', 'option_chain', 'place_forever', 'place_order', 'place_slice_order', 'quote_data', 'session', 'ticker_data', 'timeout']
Fetching 5-min data for Security ID: 26001...


In [ ]:
import os
from dhanhq import dhanhq

# Load credentials from environment variables
DHAN_CLIENT_ID = os.environ.get('DHAN_CLIENT_ID')
DHAN_ACCESS_TOKEN = os.environ.get('DHAN_ACCESS_TOKEN')

# Check if credentials are available
if not DHAN_CLIENT_ID or not DHAN_ACCESS_TOKEN:
    raise ValueError("DHAN_CLIENT_ID and DHAN_ACCESS_TOKEN must be set in environment variables")

# Initialize Dhan client
dhan = dhanhq(DHAN_CLIENT_ID, DHAN_ACCESS_TOKEN)

# Example: Get a list of all your orders
order_list = dhan.get_order_list()
print(order_list)

{'status': 'failure', 'remarks': {'error_code': 'DH-901', 'error_type': 'Invalid_Authentication', 'error_message': 'Client ID or user generated access token is invalid or expired.'}, 'data': {'errorType': 'Invalid_Authentication', 'errorCode': 'DH-901', 'errorMessage': 'Client ID or user generated access token is invalid or expired.'}}
